In [451]:
from urllib.request import urlopen, Request
import requests
from bs4 import BeautifulSoup
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [4]:
headers = {
    'authority': 'finviz.com',
    'cache-control': 'max-age=0',
    'sec-ch-ua': '"Google Chrome";v="95", "Chromium";v="95", ";Not A Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'sec-fetch-site': 'none',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-user': '?1',
    'sec-fetch-dest': 'document',
    'accept-language': 'nl-BE,nl-NL;q=0.9,nl;q=0.8,en-US;q=0.7,en;q=0.6'}

In [18]:
'''loop to get full table per ticker'''
news_tables = {}
financial = {}
tickers = ['TDOC','RDS-A', 'EURN']

for ticker in tickers:
    print(f"sending request for {ticker}")
    params = (
    ('t', ticker),
    )
    response = requests.get('https://finviz.com/quote.ashx', headers=headers, params=params) 
    # Read the contents of the file into 'html'
    html = BeautifulSoup(response.content, "html.parser")
    # Find 'news-table' in the Soup and load it into 'news_table'
    news_table = html.find('table', class_='fullview-news-outer').find_all('tr')
    snapshot_table = html.find('table', class_='snapshot-table2')
    # Add the table to our dictionary
    news_tables[ticker] = news_table
    financial[ticker] = snapshot_table

sending request for TDOC
sending request for RDS-A
sending request for EURN


In [196]:
'''Get final dict with news'''
new_dict = {}

# Iterate through the news
for file_name, news_table in news_tables.items():
    # Create new list per iteration
    parsed_news = []
    intermediate_dict = {}
    # Iterate through all tr tags in 'news_table'
    for x in news_table:
        # read the text from each tr tag into text
        # get text from a only
        text = x.a.get_text() 
        # splite text in the td tag into a list 
        date_scrape = x.td.text.split()
        # if the length of 'date_scrape' is 1, load 'time' as the only element

        if len(date_scrape) == 1:
            time = date_scrape[0]
            
        # else load 'date' as the 1st element and 'time' as the second    
        else:
            date = date_scrape[0]
            time = date_scrape[1]
        # Extract the ticker from the file name, get the string up to the 1st '_'  
        #ticker = file_name.split('_')[0]
        
        # Append ticker, date, time and headline as a list to the 'parsed_news' list
        parsed_news.append([date, text])
    #intermediate dictionary
    intermediate_dict['news'] = parsed_news
    # Append full list to dict
    new_dict[file_name] = intermediate_dict


In [197]:
'''Get financial dict'''
financial_dict = {}
for ticker, values in financial.items():
    table_dark_row = values.find_all('tr', class_="table-dark-row")
    values_dict = {}
    intermediate_dict = {}
    #table_dark_row = financial['TDOC'].find_all('tr', class_="table-dark-row")
    for i in table_dark_row:
        keys = i.find_all('td', class_="snapshot-td2-cp")
        values = i.find_all('td', class_="snapshot-td2")
        for keys, values in zip(keys, values):
            values_dict[keys.text] =  values.text
    intermediate_dict['fundamentals'] = values_dict
    new_dict[ticker]['fundamentals'] = values_dict


In [459]:
'''Clean data''''
def clean_data(data):
    #filling empty cells with nan value and treating '52W Range' column
    #mymap = {'-': float('nan')}
    range_df = data['52W Range']
    data.drop(columns=['52W Range'], axis=1, inplace=True)
    #data.applymap(lambda s: mymap.get(s) if s in mymap else s)
    
    range_df = range_df.apply(lambda x: x.split(' - '))
    data['52W Range Low'] = range_df.apply(lambda x: x[0])
    data['52W Range High'] = range_df.apply(lambda x: x[1])
    #treating % and Million/Billion values to numeric
    mymap = {'%': 0.01, 'M': 1000000, 'B': 1000000000}
    metrics_tonumeric = ['Market Cap', 'Dividend %', 'Shs Outstand']
    for metric in metrics_tonumeric:
        data.loc[:, metric] = pd.to_numeric(data[metric].str[:-1]) * \
                                    data[metric].str[-1].replace(mymap)
    metrics_tonumeric2 = ['EPS next 5Y', 'Insider Own', 
                         'ROE', 'ROI', 'ROA', 'Profit Margin']
    empty_metric = []
    for metric in metrics_tonumeric2:
        empty_metric.append(f"{metric} %")
    
    for new, old in zip(empty_metric, metrics_tonumeric2):
        data[new] = pd.to_numeric(data[old].str[:-1]) * \
                                    data[old].str[-1].replace(mymap)
    #transforming whole df to numeric values
    #data = data.apply(pd.to_numeric, errors='coerce')
    #adjustments on dataframe to make it more readable
    data['Market Cap(B)'] = data['Market Cap'].apply(lambda x: x/1000000000)
    data['Shs Outstand(M)'] = data['Shs Outstand'].apply(lambda x: x/1000000)
    data.replace('-', np.nan, inplace=True)
    columns = ['Market Cap(B)', 'Price', 'P/B', 'P/E', 'Forward P/E', 'P/S', 'PEG', 
                         'P/FCF', 'Debt/Eq', 'EPS (ttm)', 'EPS next 5Y %', 'Dividend %', 
                         'Insider Own %', 'ROE %', 'ROI %', 'ROA %', 'Profit Margin %', 
                         'Shs Outstand(M)', 'RSI (14)', 'Beta', '52W Range Low', '52W Range High']
    return data[columns] 

In [460]:
data = pd.DataFrame.from_records(new_dict['TDOC']['fundamentals'], index=['tdoc'])

In [461]:
test = clean_data(data)
test

,Market Cap(B),Price,P/B,P/E,Forward P/E,P/S,PEG,P/FCF,Debt/Eq,EPS (ttm),...,Insider Own %,ROE %,ROI %,ROA %,Profit Margin %,Shs Outstand(M),RSI (14),Beta,52W Range Low,52W Range High
tdoc,21.7,136.42,1.36,NaN,NaN,11.66,NaN,NaN,0.08,-5.76,...,0.005,-0.051,-0.025,-0.046,-0.436,159.44,43.73,0.32,120.67,308.00
